In [1]:
import numpy as np 
import os
import skimage.io as io
import skimage.transform as trans
import sklearn.metrics as sm
import tensorflow as tf
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras import backend as keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import scipy.misc as sc

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
PATH = '/content/drive/MyDrive/STARE/'
TRAIN_PATH = PATH+'train/'
TRAIN_AUG_PATH = TRAIN_PATH+'aug/'
TRAIN_IMAGE_FOLDER = 'images'
TRAIN_MASK_FOLDER = 'GT'
IMAGE_PREFIX = 'image'
MASK_PREFIX = 'mask'
TRAIN_TARGET_SIZE = (512,512)
TRAIN_BATCH_SIZE = 3
SEED = 1
TEST_PATH = PATH+'test/'
TEST_IMAGE_PATH = TEST_PATH+'images/'
SAVE_TEST_IMAGE_PATH = TEST_PATH+'pred/'
TEST_TARGET_SIZE = (512,512)

In [4]:
def adjustData(img,mask,flag_multi_class,num_class):
    if(flag_multi_class):
        img /= 255
        mask = mask[:,:,:,0] if(len(mask.shape) == 4) else mask[:,:,0]
        new_mask = np.zeros(mask.shape + (num_class,))
        for i in range(num_class):
            new_mask[mask == i,i] = 1
        new_mask = np.reshape(new_mask,(new_mask.shape[0]*new_mask.shape[1],new_mask.shape[2]))
        mask = new_mask
    elif(np.max(img) > 1):
        img = img / 255
        mask = mask /255
        mask[mask > 0.5] = 1
        mask[mask <= 0.5] = 0
    return (img,mask)

def trainGenerator(aug_dict, flag_multi_class = False, num_class = 2):
    image_datagen = ImageDataGenerator(**aug_dict)
    mask_datagen = ImageDataGenerator(**aug_dict)
    image_generator = image_datagen.flow_from_directory(
        TRAIN_PATH,
        classes = [TRAIN_IMAGE_FOLDER],
        class_mode = None,
        color_mode = 'grayscale',
        target_size = TRAIN_TARGET_SIZE,
        batch_size = TRAIN_BATCH_SIZE,
        save_to_dir = TRAIN_AUG_PATH,
        save_prefix  = IMAGE_PREFIX,
        seed = SEED)
    mask_generator = mask_datagen.flow_from_directory(
        TRAIN_PATH,
        classes = [TRAIN_MASK_FOLDER],
        class_mode = None,
        color_mode = 'grayscale',
        target_size = TRAIN_TARGET_SIZE,
        batch_size = TRAIN_BATCH_SIZE,
        save_to_dir = TRAIN_AUG_PATH,
        save_prefix  = MASK_PREFIX,
        seed = SEED)
    train_generator = zip(image_generator, mask_generator)
    for (img,mask) in train_generator:
        img,mask = adjustData(img,mask,flag_multi_class,num_class)
        yield (img,mask)

In [5]:
data_gen_args = dict(rotation_range=0.3,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.1,
                    # zoom_range=[0.7,1],
                    horizontal_flip=True,
                    fill_mode='nearest')

if not os.path.exists(TRAIN_PATH+'aug'):
    os.makedirs(TRAIN_PATH+'aug')
      
data_gen = trainGenerator(aug_dict=data_gen_args)

In [6]:
def unet(pretrained_weights = None,input_size = (512,512,1)):
    inputs = tf.keras.Input(shape=input_size)
    conv1 = Conv2D(64, 3, activation = 'relu', dilation_rate=2,padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = BatchNormalization()(conv1)
    conv1 = Conv2D(64, 3, activation = 'relu', dilation_rate=2,padding = 'same', kernel_initializer = 'he_normal')(conv1)
    conv1 = BatchNormalization()(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(128, 3, activation = 'relu', dilation_rate=2,padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = BatchNormalization()(conv2)
    conv2 = Conv2D(128, 3, activation = 'relu', dilation_rate=2, padding = 'same', kernel_initializer = 'he_normal')(conv2)
    conv2 = BatchNormalization()(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = BatchNormalization()(conv3)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    conv3 = BatchNormalization()(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = BatchNormalization()(conv4)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    conv4 = BatchNormalization()(conv4)
    drop4 = Dropout(0.5)(conv4, training=True)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = BatchNormalization()(conv5)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    conv5 = BatchNormalization()(conv5)
    drop5 = Dropout(0.5)(conv5, training=True)

    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    merge6 = concatenate([drop4,up6], axis = 3)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)
    

    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)
    

    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)
    

    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
   
    conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)

    model = tf.keras.Model(inputs = inputs, outputs = conv10)

    model.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])
    

    if(pretrained_weights):
    	model=keras.models.load_model(pretrained_weights)

    return model

In [7]:
model = unet()
model_checkpoint = tf.keras.callbacks.ModelCheckpoint('unet_STARE.hdf5', monitor='loss', verbose=1, save_best_only=True)
callbacks = [ model_checkpoint ]
history = model.fit(
    data_gen,
    steps_per_epoch=10,
    epochs=80,
    verbose=0,
    callbacks=callbacks)

Found 10 images belonging to 1 classes.
Found 10 images belonging to 1 classes.

Epoch 1: loss improved from inf to 0.66791, saving model to unet_STARE.hdf5

Epoch 2: loss did not improve from 0.66791

Epoch 3: loss did not improve from 0.66791

Epoch 4: loss did not improve from 0.66791

Epoch 5: loss did not improve from 0.66791

Epoch 6: loss did not improve from 0.66791

Epoch 7: loss improved from 0.66791 to 0.66672, saving model to unet_STARE.hdf5

Epoch 8: loss improved from 0.66672 to 0.66277, saving model to unet_STARE.hdf5

Epoch 9: loss improved from 0.66277 to 0.65868, saving model to unet_STARE.hdf5

Epoch 10: loss improved from 0.65868 to 0.65482, saving model to unet_STARE.hdf5

Epoch 11: loss improved from 0.65482 to 0.65082, saving model to unet_STARE.hdf5

Epoch 12: loss improved from 0.65082 to 0.64697, saving model to unet_STARE.hdf5

Epoch 13: loss improved from 0.64697 to 0.64304, saving model to unet_STARE.hdf5

Epoch 14: loss improved from 0.64304 to 0.63935, sa

[GPU]
start aug: 11.16
train: 15 menit

In [8]:
acc = history.history['accuracy']
loss = history.history['loss']

max_acc = max(acc)
max_acc_i = acc.index(max_acc)
loss_at_max_acc = loss[max_acc_i]

print('acc:', max_acc, 'loss:', loss_at_max_acc)

acc: 0.9262034296989441 loss: 0.4923255741596222


14-02-2023; 23:40 <br>
acc: 0.9706441164016724 loss: 0.09779315441846848 <br><br>

15-02-2023; 01:53 <br>
acc: 0.9781307578086853 loss: 0.06890685111284256 <br>
change: remove zoom scale <br><br>

15-02-2023; 10:16 <br>
acc: 0.9769975543022156 loss: 0.07307402044534683 <br><br>

15-02-2023; 23:32 <br>
acc: 0.9262034296989441 loss: 0.4923255741596222 <br><br>

In [9]:
acc = history.history['accuracy']
loss = history.history['loss']

max_acc = max(acc)
max_acc_i = acc.index(max_acc)
loss_at_max_acc = loss[max_acc_i]

print('acc:', max_acc, 'loss:', loss_at_max_acc)

acc: 0.9262034296989441 loss: 0.4923255741596222


In [10]:
def testGenerator(as_gray = True):
    files=sorted(os.listdir(TEST_IMAGE_PATH))
    num_image=len(files)
    for i in range(num_image):
        img = io.imread(os.path.join(TEST_IMAGE_PATH,files[i]),as_gray = as_gray)
        print(files[i])
        img = trans.resize(img,TEST_TARGET_SIZE)
        img = np.reshape(img,img.shape+(1,))
        img = np.reshape(img,(1,)+img.shape)
        yield img

In [11]:
def labelVisualize(num_class,color_dict,img):
    img = img[:,:,0] if len(img.shape) == 3 else img
    img_out = np.zeros(img.shape + (3,))
    for i in range(num_class):
        img_out[img == i] = color_dict[i]
      
    return img_out

def saveResult(npyfile):
    files=os.listdir(TEST_IMAGE_PATH)
    
    for i,item in enumerate(npyfile):
        img = item[:,:,0]
        img[img>0.1]=1
        img[img<=0.1]=0
        io.imsave(os.path.join(SAVE_TEST_IMAGE_PATH, files[i]+'_predict.png'),img)

if not os.path.exists(SAVE_TEST_IMAGE_PATH):
    os.makedirs(SAVE_TEST_IMAGE_PATH)

In [12]:
n_i = len(os.listdir(TEST_IMAGE_PATH))
test_gen = testGenerator()
results = model.predict_generator(test_gen,n_i,verbose=1)
saveResult(results)

<ipython-input-12-046c9d728f71>:3: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  results = model.predict_generator(test_gen,n_i,verbose=1)


im0162.ppm
10/10 [==============================] - 5s 472ms/step


<ipython-input-11-f97644e6b6be>:16: UserWarning: /content/drive/MyDrive/STARE/test/pred/im0291.ppm_predict.png is a low contrast image
  io.imsave(os.path.join(SAVE_TEST_IMAGE_PATH, files[i]+'_predict.png'),img)
<ipython-input-11-f97644e6b6be>:16: UserWarning: /content/drive/MyDrive/STARE/test/pred/im0163.ppm_predict.png is a low contrast image
  io.imsave(os.path.join(SAVE_TEST_IMAGE_PATH, files[i]+'_predict.png'),img)
<ipython-input-11-f97644e6b6be>:16: UserWarning: /content/drive/MyDrive/STARE/test/pred/im0235.ppm_predict.png is a low contrast image
  io.imsave(os.path.join(SAVE_TEST_IMAGE_PATH, files[i]+'_predict.png'),img)
<ipython-input-11-f97644e6b6be>:16: UserWarning: /content/drive/MyDrive/STARE/test/pred/im0162.ppm_predict.png is a low contrast image
  io.imsave(os.path.join(SAVE_TEST_IMAGE_PATH, files[i]+'_predict.png'),img)
<ipython-input-11-f97644e6b6be>:16: UserWarning: /content/drive/MyDrive/STARE/test/pred/im0236.ppm_predict.png is a low contrast image
  io.imsave(os.pa

In [13]:
def get_confusion_matrix_elements(groundtruth_list, predicted_list):
    tn, fp, fn, tp = sm.confusion_matrix(groundtruth_list, predicted_list,labels=[0,1]).ravel()
    tn, fp, fn, tp = np.float64(tn), np.float64(fp), np.float64(fn), np.float64(tp)

    return tn, fp, fn, tp

def get_prec_rec_IoU_accuracy(groundtruth_list, predicted_list):
    tn, fp, fn, tp = get_confusion_matrix_elements(groundtruth_list, predicted_list)
    
    total = tp + fp + fn + tn
    accuracy = (tp + tn) / total
    prec = tp/(tp+fp)
    rec = tp/(tp+fn)
    IoU = tp/(tp+fp+fn)
    
    return prec,rec,IoU,accuracy

def get_f1_score(groundtruth_list, predicted_list):
    tn, fp, fn, tp = get_confusion_matrix_elements(groundtruth_list, predicted_list)
    
    f1_score = (2 * tp) / ((2 * tp) + fp + fn)

    return f1_score

def get_validation_metrics(groundtruth,predicted):   
    u,v = np.shape(groundtruth)
    groundtruth_list = np.reshape(groundtruth,(u*v,))
    predicted_list = np.reshape(predicted,(u*v,))
    prec,rec,IoU,acc = get_prec_rec_IoU_accuracy(groundtruth_list, predicted_list)
    f1_score = get_f1_score(groundtruth_list, predicted_list)

    return prec,rec,IoU,acc,f1_score

def evalResult(gth_path,npyfile,target_size=(512,512),flag_multi_class = False,num_class = 2):
    files=sorted(os.listdir(gth_path))
    print(files)
    prec = 0
    rec = 0
    acc = 0
    IoU = 0
    f1_score=0
    for i,item in enumerate(npyfile):
        img = item[:,:,0]
        gth = io.imread(os.path.join(gth_path,files[i]))
        gth = trans.resize(gth,target_size)
        img1=np.array(((img - np.min(img))/np.ptp(img))>0.1).astype(float)
        gth1=np.array(((gth - np.min(gth))/np.ptp(gth))>0.1).astype(float)
        p,r,I,a,f=get_validation_metrics(gth1,img1)
        prec = prec+p
        rec = rec+r
        acc = acc+a
        IoU = IoU+I
        f1_score = f1_score+f
    print("Precision=",prec/(i+1), "Recall=",rec/(i+1), "IoU=",IoU/(i+1), "acc=",acc/(i+1), "F1=",f1_score/(i+1))    

In [14]:
gt_path = TEST_PATH + 'GT/'
evalResult(gt_path,results)

['im0162.ah.ppm', 'im0163.ah.ppm', 'im0235.ah.ppm', 'im0236.ah.ppm', 'im0239.ah.ppm', 'im0240.ah.ppm', 'im0255.ah.ppm', 'im0291.ah.ppm', 'im0319.ah.ppm', 'im0324.ah.ppm']
Precision= 0.09468648790927088 Recall= 1.0 IoU= 0.09468648790927088 acc= 0.09468994140625 F1= 0.17229373847296356


### Predicted
Precision= 0.7486418535745212 Recall= 0.8802602326917179 IoU= 0.6784592658474139 acc= 0.9618434906005859 F1= 0.8066515398308723 

<b>15-02-2023; 10:30</b> 
Precision= 0.6528929903566489 Recall= 0.902777067006806 IoU= 0.6070497388289681 acc= 0.9461292266845703 F1= 0.7534104948419812

<b>15-02-2023; 22:33</b> 
Precision= 0.09468648790927088 Recall= 1.0 IoU= 0.09468648790927088 acc= 0.09468994140625 F1= 0.17229373847296356